In [1]:
import requests
import pandas as pd
import numpy as np
from dotenv import dotenv_values
import geopandas as gpd
config = dotenv_values(".env") #apikey



Great Documentation can be found here: https://buildmedia.readthedocs.org/media/pdf/postgrest/v0.3.2/postgrest.pdf

We start by register the Station data with the database.

In [2]:
station = {
    "stationname":"sniffer",
    "projectid":"SETx-UIFL Beaumont",
    "description": "Beaumont Run of the SNIFFER air quality sensor for VOCUS data",
    "contactname": "Pawell",
    "contactemail": "Pawell@utexas.edu",
    "active":True,
    "startdate": "Feb 23, 2023 17:05:57",
    "datetime": "Feb 25, 2023 12:18:11"
}
# r = requests.post("https://postgrest-dev.proudflower-a6582e11.centralus.azurecontainerapps.io/station", headers={'Authorization': f'Bearer {config["apikey"]}',
#          }, data=station)
# print(r)

We can then  clean the sensor data. which requires us to load 4 datasets.
 - Mass list which will generate each measurements alias.
- Sensor provides the 1d measurement data from the Sniffer
- engdata provides the 1d engineering metadata from the sensor
- engdataNames is the header file for the engdata

In [3]:
MassList = pd.read_csv("2023Beaumont_partial/w_data/Export/UL/IonList/MassIDs_2023.02.23-17h36m01sUL.csv", header=None)
sensor = np.fromfile("2023Beaumont_partial/w_data/Export/UL/CPS/cps2023.02.23-17h36m01sUL.fdt", sep="\n")
engData = np.fromfile("2023Beaumont_partial/w_data/Export/EngData/EngData2023.02.23-17h36m01s.fdt",sep="\n")
with open ("2023Beaumont_partial/w_data/Export/EngData/EngDataNames2023.02.23-17h36m01s.csv" , encoding="ISO-8859-1") as f:
    engDataNames = (f.read().split("\n"))

For the sensor data we need to add columns that are the same for each.

In [4]:
sensor_table = pd.DataFrame({"alias":MassList[0].tolist()})
sensor_table['postprocess']=True
sensor_table['postprocessscript']=None
sensor_table['units']='Counts Per Second'
sensor_table['datatype']=1


For the Sensor data it is linked to a specific station. Which can be found by the project name SETx-UIFL. Below I'll make a request that filters out by SETx-UIFL and the the station Name sniffer.

To use filters we will use the column name =eq.  and the the specific name
ex :  projectid=eq.SETx-UIFL Beaumont


the following operators are available:

|  abbreviation |  meaning|
|---------------|---------|
|  eq | equals | 
|  gte | greater than or equal | 
| gt | greater than | 
| lte | less than or equal | 
| lt | less than | 
| neq | not equal | 
| like | LIKE operator (use * in place of %) | 
| ilike | ILIKE operator (use * in place of %) | 
| in | one of a list of values e.g. ?a=in.1,2,3 | 
| notin|  not one of a list of values e.g. ?a=notin.1,2,3 is checking for exact equality (null,true,false) | 
| isnot | checking for exact inequality (null,true,false) | 
| @@ | full-text search using to_tsquery | 
| @> | contains e.g. ?tags=@>.{example, new} <@ contained in e.g. values=<@{1,2,3} | 
| not | negates another operator, see below|

In [5]:
# r = requests.get("https://postgrest-dev.proudflower-a6582e11.centralus.azurecontainerapps.io/station?projectid=eq.SETx-UIFL Beaumont&stationname=eq.sniffer")
# sensor_table['stationid']= r.json()[0]['stationid']

Then we can upload that sensor table into the database.

In [6]:
# r = requests.post("https://postgrest-dev.proudflower-a6582e11.centralus.azurecontainerapps.io/sensor", headers={'Authorization': f'Bearer {config["apikey"]}',
#          'Content-Type':'text/csv'}, data=sensor_table.to_csv(header=True, index=False))

Finally we will transform the measurement data into a useable format and upload the data.

We'll start by transforming the engData and measurement_df from 1-Dimensional to 2-dimensional. We skip the first three lines because they are the header to the file.

In [7]:
engData = pd.DataFrame(engData[3:].reshape(int(engData[1]),int(engData[2]), order='F' ), columns=engDataNames[:-1])
measurement_df = pd.DataFrame(sensor[3:].reshape(int(sensor[2]), int(sensor[1])), columns=(MassList[0].tolist()))

We then create the collection time for each measurement. This is based on the engData JulianDate Column. Julian Date is the number of dates since Jan 01, 2009.

In [8]:
engData['collectiontime']=(pd.to_datetime(pd.Timestamp('2009-01-01T00:00:00').to_julian_date()+engData['JulianDate'], unit='D',origin='julian'))


In [9]:
measurement_df=(pd.concat([engData, measurement_df], axis=1))

In [10]:

# measurement_df['elevation']=None
# measurement_df['geometry']=None

The Last thing we'll need to add to the measurement is the GPS locations.

In [11]:
gps = pd.read_csv("2023Beaumont_partial/Beaumont2023FebMarchOutputGPS.txt")
gps['Time_CDT']=(pd.to_datetime(gps['Time_CDT']))
print(gps.loc[(gps['Time_CDT'].dt.month==2) & (gps['Time_CDT'].dt.day==23) & (gps['Time_CDT'].dt.hour==17)& (gps['Time_CDT'].dt.minute==49) ])

                      Time_UTC            Time_CDT    Lat_deg   Lon_deg  \
33077  2023/02/23 22:49:00.000 2023-02-23 17:49:00  30.046037 -94.07243   
33078  2023/02/23 22:49:01.000 2023-02-23 17:49:01  30.046037 -94.07243   
33079  2023/02/23 22:49:02.000 2023-02-23 17:49:02  30.046037 -94.07243   
33080  2023/02/23 22:49:03.000 2023-02-23 17:49:03  30.046037 -94.07243   
33081  2023/02/23 22:49:04.000 2023-02-23 17:49:04  30.046037 -94.07243   
33082  2023/02/23 22:49:05.000 2023-02-23 17:49:05  30.046037 -94.07243   
33083  2023/02/23 22:49:06.000 2023-02-23 17:49:06  30.046037 -94.07243   
33084  2023/02/23 22:49:07.000 2023-02-23 17:49:07  30.046037 -94.07243   
33085  2023/02/23 22:49:08.000 2023-02-23 17:49:08  30.046037 -94.07243   
33086  2023/02/23 22:49:09.000 2023-02-23 17:49:09  30.046037 -94.07243   
33087  2023/02/23 22:49:10.000 2023-02-23 17:49:10  30.046037 -94.07243   
33088  2023/02/23 22:49:11.000 2023-02-23 17:49:11  30.046037 -94.07243   
33089  2023/02/23 22:49:1

We can see there will be a mismatch between the time GPS data is collected and sensor data. So have a few steps to clean the data before we can link the GPS and measurement data:
  - Use the get point at Distance to interpolate the new lat long.
    - We need to convert Speed m_s to km_s and then calculate the distance as delta between time gps and sensor.
    - *Code from: https://stackoverflow.com/questions/7222382/get-lat-long-given-current-point-distance-and-bearing*
- convert the GPS dataframe to Geopandas dataframe and a geometry point.


In [12]:
from math import asin, atan2, cos, degrees, radians, sin

def get_point_at_distance(lat1, lon1, d, bearing, R=6371):
    """ https://stackoverflow.com/questions/7222382/get-lat-long-given-current-point-distance-and-bearing
    lat: initial latitude, in degrees
    lon: initial longitude, in degrees
    d: target distance from initial
    bearing: (true) heading in degrees
    R: optional radius of sphere, defaults to mean radius of earth

    Returns new lat/lon coordinate {d}km from initial, in degrees
    """
    lat1 = radians(lat1)
    lon1 = radians(lon1)
    a = radians(bearing)
    lat2 = asin(sin(lat1) * cos(d/R) + cos(lat1) * sin(d/R) * cos(a))
    lon2 = lon1 + atan2(
        sin(a) * sin(d/R) * cos(lat1),
        cos(d/R) - sin(lat1) * sin(lat2)
    )
    return (degrees(lat2), degrees(lon2),)

In [13]:
def calculate_lat_long(t):
    subset = gps.loc[(gps['Time_CDT'].dt.year==t.year) & (gps['Time_CDT'].dt.month==t.month) & (gps['Time_CDT'].dt.day==t.day)
 & (gps['Time_CDT'].dt.hour==t.hour)& (gps['Time_CDT'].dt.minute==t.minute)& (gps['Time_CDT'].dt.second==t.second), ]
    try:
        delta = (t.microsecond - subset['Time_CDT'].dt.microsecond.to_list()[0]) * .000001

        lat = subset['Lat_deg'].to_list()[0]
        long = subset['Lon_deg'].to_list()[0]
        degree = subset['Heading_deg'].to_list()[0]
        distance = delta * subset['Speed m_s'].to_list()[0]
    except:
        subset = gps.loc[(gps['Time_CDT'].dt.year==t.year) & (gps['Time_CDT'].dt.month==t.month) & (gps['Time_CDT'].dt.day==t.day)
 & (gps['Time_CDT'].dt.hour==t.hour)& (gps['Time_CDT'].dt.minute==t.minute)& (gps['Time_CDT'].dt.second==t.second-1), ]
        delta = (t.microsecond - subset['Time_CDT'].dt.microsecond.to_list()[0]) * .000001

        lat = subset['Lat_deg'].to_list()[0]
        long = subset['Lon_deg'].to_list()[0]
        degree = subset['Heading_deg'].to_list()[0]
        distance = delta * subset['Speed m_s'].to_list()[0]
    lat, long = get_point_at_distance(lat, long, distance, degree)
    return  f'POINT({lat} {long})'

In [14]:
measurement_df['geometry']=measurement_df['collectiontime'].apply(lambda row: calculate_lat_long(row) )

In [15]:
print(measurement_df['geometry'])

0        POINT(30.046026491635423 -94.0731252314149)
1       POINT(30.023292515520765 -94.06698123834101)
2       POINT(30.025708174510278 -94.06359943652627)
3       POINT(30.045600328479882 -94.07271915374736)
4       POINT(30.028833898418693 -94.06001808846878)
                            ...                     
1795     POINT(30.00728179898451 -94.03820809236207)
1796    POINT(30.012291644596665 -94.04249124627863)
1797    POINT(30.009754937238696 -94.03976809505963)
1798     POINT(30.01439183827795 -94.04449054528398)
1799     POINT(30.01094466023334 -94.04086961772742)
Name: geometry, Length: 1800, dtype: object


In [18]:
measurement_df.to_csv('measurement2.csv', index=False)

In [16]:
for m in MassList[0].tolist():
    print(m)

    # r = requests.get(f"https://postgrest-dev.proudflower-a6582e11.centralus.azurecontainerapps.io/sensor?alias=eq.{m}")
    # measurement_df["sensorid"]=r.json()[0]['sensorid']
    measurement_df['measurementvalue']=measurement_df[m]
    data = measurement_df[['sensorid','collectiontime', 'geometry',  'measurementvalue' ]].to_csv(header=True, index=False)
    # r=requests.post("https://postgrest-dev.proudflower-a6582e11.centralus.azurecontainerapps.io/measurement", headers={'Authorization': f'Bearer {config["apikey"]}',
        #  'Content-Type':'text/csv'}, data=data)




12.3623


NameError: name 'r' is not defined